In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score #since the metric used by the evaluator is r2_score
from sklearn.preprocessing import LabelEncoder #gonna use it to deal with the strings in the dataset
from sklearn.preprocessing import MinMaxScaler #for normalization
from sklearn.svm import LinearSVR #gonna use Linear SVM as my one of the regression methods
from collections import Counter


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/DSL

/content/drive/MyDrive/DSL


In [4]:
ls

 dataset/  'DS_Lab(summer).ipynb'   Ridge.csv   Ridge_final.csv


In [5]:
dataset=pd.read_csv('dataset/development.tsv',sep='\t') #importing the dataset
eval_dataset=pd.read_csv('dataset/evaluation.tsv',sep='\t')

In [6]:
#gaining information about the dataset
print(type(dataset))
print('====================\n')
print(dataset.head())
print('====================\n')
dataset.info()

<class 'pandas.core.frame.DataFrame'>

   beer/ABV  ... user/profileName
0      5.70  ...       goblue3509
1      9.99  ...      Lounge14ABV
2      5.35  ...        DogFood11
3      8.00  ...  BasementDweller
4      7.00  ...      OneDropSoup

[5 rows x 14 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   beer/ABV           66893 non-null  float64
 1   beer/name          70000 non-null  object 
 2   beer/style         70000 non-null  object 
 3   review/appearance  70000 non-null  float64
 4   review/aroma       70000 non-null  float64
 5   review/overall     70000 non-null  float64
 6   review/palate      70000 non-null  float64
 7   review/taste       70000 non-null  float64
 8   review/text        69982 non-null  object 
 9   user/ageInSeconds  14645 non-null  float64
 10  user/birthdayRaw   14645 non-null  object 


In [7]:
dataset

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,5.70,Barney Flats Oatmeal Stout,Oatmeal Stout,4.5,5.0,4.5,4.0,4.5,Pours a dark almost cola color with reddish br...,1.066918e+09,"Feb 19, 1981",351417600.0,Male,goblue3509
1,9.99,Brown Shugga',American Strong Ale,4.0,4.0,4.0,4.5,5.0,I went to a place called World Market in Oxnar...,NaN,NaN,NaN,NaN,Lounge14ABV
2,5.35,Casta Bruna (Pale Ale),English Pale Ale,3.0,3.0,4.0,3.0,3.5,I clicked on Casta tonight just to add some no...,NaN,NaN,NaN,NaN,DogFood11
3,8.00,Old Engine Oil Special Reserve (Aged In Malt W...,Old Ale,4.0,4.5,4.0,5.0,4.0,Wonderful little beer. Pours a deep brown with...,NaN,NaN,NaN,NaN,BasementDweller
4,7.00,Dark Humor,American Porter,3.5,4.0,4.0,4.0,4.5,Excited to see this on tap at Fathead's origin...,NaN,NaN,NaN,NaN,OneDropSoup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,4.70,Lvivske Premium,Euro Pale Lager,2.0,2.5,2.5,3.0,2.0,Thanks to witbier for the bottle...\tAppears a...,1.215789e+09,"Jun 2, 1976",202546800.0,Male,weeare138
69996,5.01,Samuel Adams East-West KÃ¶lsch,KÃ¶lsch,4.0,3.5,4.0,3.5,3.5,Poured into a glass as close to a stange that ...,1.359900e+09,"Nov 8, 1971",58435200.0,Male,ujsplace
69997,NaN,Sam The Man,American Strong Ale,3.5,3.5,3.5,3.0,3.5,This beer is basically a blend of the brewerie...,NaN,NaN,NaN,NaN,beertunes
69998,6.00,Chatoe Rogue First Growth Dirtoir Black Lager,Schwarzbier,4.0,4.0,4.0,4.0,4.0,"Poured a delightfully dense black, perhaps the...",NaN,NaN,NaN,NaN,Brent


In [8]:
#now we have to seperate the class labels from samples 
y=dataset.iloc[:,5]
X=dataset.loc[:, dataset.columns != 'review/overall']
print(X.info())
print('\n====================\n')
print(y.head())
print('====================\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   beer/ABV           66893 non-null  float64
 1   beer/name          70000 non-null  object 
 2   beer/style         70000 non-null  object 
 3   review/appearance  70000 non-null  float64
 4   review/aroma       70000 non-null  float64
 5   review/palate      70000 non-null  float64
 6   review/taste       70000 non-null  float64
 7   review/text        69982 non-null  object 
 8   user/ageInSeconds  14645 non-null  float64
 9   user/birthdayRaw   14645 non-null  object 
 10  user/birthdayUnix  14645 non-null  float64
 11  user/gender        28181 non-null  object 
 12  user/profileName   69986 non-null  object 
dtypes: float64(7), object(6)
memory usage: 6.9+ MB
None


0    4.5
1    4.0
2    4.0
3    4.0
4    4.0
Name: review/overall, dtype: float64



In [9]:
#label encoder is used for encoding the usefull categorical attributes such as beer/name or beer/style

In [10]:
X.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/text,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,5.70,Barney Flats Oatmeal Stout,Oatmeal Stout,4.5,5.0,4.0,4.5,Pours a dark almost cola color with reddish br...,1.066918e+09,"Feb 19, 1981",351417600.0,Male,goblue3509
1,9.99,Brown Shugga',American Strong Ale,4.0,4.0,4.5,5.0,I went to a place called World Market in Oxnar...,NaN,NaN,NaN,NaN,Lounge14ABV
2,5.35,Casta Bruna (Pale Ale),English Pale Ale,3.0,3.0,3.0,3.5,I clicked on Casta tonight just to add some no...,NaN,NaN,NaN,NaN,DogFood11
3,8.00,Old Engine Oil Special Reserve (Aged In Malt W...,Old Ale,4.0,4.5,5.0,4.0,Wonderful little beer. Pours a deep brown with...,NaN,NaN,NaN,NaN,BasementDweller
4,7.00,Dark Humor,American Porter,3.5,4.0,4.0,4.5,Excited to see this on tap at Fathead's origin...,NaN,NaN,NaN,NaN,OneDropSoup


In [11]:
#now we have to use label encoder to handle the categorical data
#we can drop some useless columns as well like user/ageInSeconds	user/birthdayRaw	user/birthdayUnix	user/gender	

In [12]:
print('number of NaN values in some of the columns')
print('=================')

print(X['user/gender'].isna().sum())
print('=================')

print(X['beer/name'].isna().sum())
print('=================')

print(X['beer/style'].isna().sum())
print('=================')

print(X['review/text'].isna().sum())
print('=================')

print(X['review/taste'].isna().sum())
print('=================')

print(X['user/birthdayUnix'].isna().sum())
print('=================')

print(X['user/birthdayRaw'].isna().sum())
print('=================')

print(X['user/ageInSeconds'].isna().sum())
print('=================')

print(X['review/appearance'].isna().sum())
print('=================')

print(X['review/aroma'].isna().sum())
print('=================')

print(X['beer/ABV'].isna().sum())
print('=================')

print(X['user/profileName'].isna().sum())
print('=================')

print(X['review/palate'].isna().sum())

number of NaN values in some of the columns
41819
0
0
18
0
55355
55355
55355
0
0
3107
14
0


In [13]:

print(y.isna().sum())
print('=================')

0


In [14]:
#here we drop the useless columns
X=X.drop(columns=['user/gender', 'user/birthdayUnix', 'user/ageInSeconds', 'user/birthdayRaw'])

In [15]:
#here we get the columns we want to use label encoding on
#it will be so hard to use reviews and we have to use techniques like word embeding 
#we can't use the gender coulumn because it has too many NaN values
#for the beer/ABV	 column we use the mean value to replace the NaN values. we don't drop this column cause it only contains 3107 values which is nothing compared to 70000 entries
beer_name=X['beer/name']
#in the later parts, the  model label_encoder_beer_name_classes says that hasn't seen some labels
#here we add "UNK" to our data so that we can replace unknown values with "UNK", cause now our model can be trained with the values of "UNK"
beer_name = beer_name.append(pd.Series('UNK'),ignore_index=True)
beer_style=X['beer/style']
profile_name=X['user/profileName']
profile_name = profile_name.replace(np.nan, 'UNK')
beerABV=X['beer/ABV']
beerABV = beerABV.replace(np.nan,beerABV.mean()) #we replace the NaN values in this column with the "mean" value

# profile_name = profile_name.fillna('UNK') #to replace the NaN values with something else 

In [16]:
#now we have to perfomre word embeding on the selected columns
label_encoder_profile_name=LabelEncoder()
label_encoder_beer_style=LabelEncoder()
label_encoder_beer_name=LabelEncoder()


label_encoder_profile_name.fit(profile_name)
labeled_profile_name=label_encoder_profile_name.transform(profile_name)

label_encoder_profile_name_classes = list(label_encoder_profile_name.classes_)
label_encoder_profile_name_num = len(label_encoder_profile_name_classes)


label_encoder_beer_style.fit(beer_style)
labeled_beer_style=label_encoder_beer_style.transform(beer_style)

label_encoder_beer_style_classes = list(label_encoder_beer_style.classes_)
label_encoder_beer_style_num = len(label_encoder_beer_style_classes)

#we perfome the fit method on the beer_name cause it contains the "UNK" value 
#but we treak the the  model and fit the model on the "UNK" containing data but we transfrom on the main data cause if we don't, we will face problems relating to the lenght of the
# of the data
label_encoder_beer_name.fit(beer_name)
labeled_beer_name=label_encoder_beer_name.transform(X['beer/name'])

label_encoder_beer_name_classes = list(label_encoder_beer_name.classes_)
label_encoder_beer_name_num = len(label_encoder_beer_name_classes)



In [17]:
#label encoding starts from 0 and it might be a problem in some algorithms so we have to increment the given values by one
#the "labeled" variables are numpy arrays
#label encoding gives weights to data but in this case we don't really want to use weights so we have to use one hot encoder but one hot will be a headache to use in this dataset
#so we go with the label encoder anyway
labeled_profile_name=[x+1 for x in labeled_profile_name]
labeled_beer_style=[x+1 for x in labeled_beer_style]
labeled_beer_name=[x+1 for x in labeled_beer_name]

In [18]:
#now we replace the data in our dataset with the labeled data 

X['user/profileName']=labeled_profile_name
X['beer/style']=labeled_beer_style
X['beer/name']=labeled_beer_name
X['beer/ABV']=beerABV

In [19]:
X.head()


,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/text,user/profileName
0,5.70,1139,84,4.5,5.0,4.0,4.5,Pours a dark almost cola color with reddish br...,6605
1,9.99,2504,20,4.0,4.0,4.5,5.0,I went to a place called World Market in Oxnar...,2495
2,5.35,2904,48,3.0,3.0,3.0,3.5,I clicked on Casta tonight just to add some no...,1232
3,8.00,9474,85,4.0,4.5,5.0,4.0,Wonderful little beer. Pours a deep brown with...,345
4,7.00,3702,18,3.5,4.0,4.0,4.5,Excited to see this on tap at Fathead's origin...,2994


In [20]:
#since we have to use word embeding for the review part, we drop the column. later on, we can use techniques such as word embeding to use this column as well
review_text_dev=X.iloc[:,7]
X=X.drop(columns=['review/text'])

In [21]:
X.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/palate,review/taste,user/profileName
0,5.70,1139,84,4.5,5.0,4.0,4.5,6605
1,9.99,2504,20,4.0,4.0,4.5,5.0,2495
2,5.35,2904,48,3.0,3.0,3.0,3.5,1232
3,8.00,9474,85,4.0,4.5,5.0,4.0,345
4,7.00,3702,18,3.5,4.0,4.0,4.5,2994


In [22]:
#now we have to normalize the data
min_max=MinMaxScaler()
min_max.fit(X[['beer/ABV','beer/name','beer/style','review/appearance','review/aroma','review/palate', 'review/taste','user/profileName']])
X[['beer/ABV','beer/name','beer/style','review/appearance','review/aroma','review/palate', 'review/taste','user/profileName']]=min_max.transform(X[['beer/ABV','beer/name','beer/style','review/appearance','review/aroma','review/palate', 'review/taste','user/profileName']])

In [23]:
X.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/palate,review/taste,user/profileName
0,0.132678,0.077048,0.805825,0.875,1.000,0.750,0.875,0.624610
1,0.238084,0.169465,0.184466,0.750,0.750,0.875,1.000,0.235884
2,0.124079,0.196547,0.456311,0.500,0.500,0.500,0.625,0.116429
3,0.189189,0.641368,0.815534,0.750,0.875,1.000,0.750,0.032536
4,0.164619,0.250575,0.165049,0.625,0.750,0.750,0.875,0.283080


Now we prepare the evaluation dataset 

In [24]:
eval_dataset.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/text,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,5.1,Roxy Rolles,American Amber / Red Ale,3.0,3.0,3.0,1.5,Disagreeing with the Bros on this one...\t\tAp...,NaN,NaN,NaN,NaN,lilbrewmonkey
1,10.0,Burton Baton,American Double / Imperial IPA,4.5,4.0,5.0,4.0,Love this brew. I'll start with a review of th...,NaN,NaN,NaN,NaN,Tripphop
2,5.6,Old #38 Stout,Irish Dry Stout,4.0,4.0,3.0,4.0,"Dark brown color, medium head. Deeply roasted ...",1.782054e+09,"Jun 23, 1958",-363718800.0,Male,beerguy101
3,7.2,Indian Brown Ale,American Brown Ale,4.5,4.5,4.0,5.0,"Pours a completely black, opaque color, with a...",NaN,NaN,NaN,Male,terrabyte
4,3.6,Greene King IPA Draught Bitter,English Bitter,3.0,2.0,2.5,2.0,Refreshingly lime-infused which is a complimen...,NaN,NaN,NaN,Male,ngandhi


In [25]:
print('number of NaN values in some of the columns')
print('=================')

print(eval_dataset['user/gender'].isna().sum())
print('=================')

print(eval_dataset['beer/name'].isna().sum())
print('=================')

print(eval_dataset['beer/style'].isna().sum())
print('=================')

print(eval_dataset['review/text'].isna().sum())
print('=================')

print(eval_dataset['review/taste'].isna().sum())
print('=================')

print(eval_dataset['user/birthdayUnix'].isna().sum())
print('=================')

print(eval_dataset['user/birthdayRaw'].isna().sum())
print('=================')

print(eval_dataset['user/ageInSeconds'].isna().sum())
print('=================')

print(eval_dataset['review/appearance'].isna().sum())
print('=================')

print(eval_dataset['review/aroma'].isna().sum())
print('=================')

print(eval_dataset['beer/ABV'].isna().sum())
print('=================')

print(eval_dataset['user/profileName'].isna().sum())
print('=================')

print(eval_dataset['review/palate'].isna().sum())

number of NaN values in some of the columns
17870
0
0
4
0
23728
23728
23728
0
0
1285
3
0


In [26]:
#for the evaluation we should fit the model 
#cause the labels used for the training set has to be the same for the test set
#now we do the same 
review_text_eval=eval_dataset.iloc[:,7]

In [27]:
eval_dataset=eval_dataset.drop(columns=['user/gender', 'user/ageInSeconds', 'user/birthdayUnix', 'user/birthdayRaw','review/text'])
 #out model doesn'e recognise some labels so we have to replace them with "UNK"

cnt = 0
eval_beer_name = []
eval_beer_style=[]
eval_profile_name=[]
for e in eval_dataset['beer/name']:
  if e not in label_encoder_beer_name_classes:
    eval_beer_name.append('UNK')
  else:
    eval_beer_name.append(e)

eval_dataset['beer/name'] = eval_beer_name

for e in eval_dataset['user/profileName']:
  if e not in label_encoder_profile_name_classes:
    eval_profile_name.append('UNK')
  else:
    eval_profile_name.append(e)

eval_dataset['user/profileName'] =eval_profile_name

for e in eval_dataset['beer/style']:
  if e not in label_encoder_beer_style_classes:
    eval_beer_style.append('UNK')
  else:
    eval_beer_style.append(e)

eval_dataset['beer/style'] = eval_beer_style

beerABV_eval=eval_dataset['beer/ABV']
beerABV_eval = beerABV_eval.replace(np.nan,beerABV_eval.mean())
eval_dataset['beer/ABV']=beerABV_eval
eval_dataset['beer/name']=label_encoder_beer_name.transform(eval_dataset['beer/name'])
eval_dataset['user/profileName']=label_encoder_profile_name.transform(eval_dataset['user/profileName'])
eval_dataset['beer/style']=label_encoder_beer_style.transform(eval_dataset['beer/style'])

In [28]:
#now we have to normalize the evalutation dataset as well 
eval_dataset[['beer/ABV','beer/name','beer/style','review/appearance','review/aroma','review/palate', 'review/taste','user/profileName']]=min_max.transform(eval_dataset[['beer/ABV','beer/name','beer/style','review/appearance','review/aroma','review/palate', 'review/taste','user/profileName']])

In [29]:
eval_dataset

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/palate,review/taste,user/profileName
0,0.117936,0.747326,0.009709,0.500,0.500,0.500,0.125,0.393928
1,0.238329,0.177116,0.077670,0.875,0.750,1.000,0.750,0.389483
2,0.130221,0.638930,0.640777,0.750,0.750,0.500,0.750,0.458621
3,0.169533,0.460325,0.058252,0.875,0.875,0.750,1.000,0.933794
4,0.081081,0.927962,0.407767,0.500,0.250,0.375,0.250,0.799678
...,...,...,...,...,...,...,...,...
29995,0.201474,0.475085,0.097087,0.750,0.500,0.500,0.500,0.205713
29996,0.140049,0.927962,0.000000,0.750,0.625,0.750,0.625,0.309468
29997,0.105651,0.043263,0.145631,0.750,0.375,0.375,0.375,0.529462
29998,0.095823,0.324848,0.766990,0.625,0.625,0.625,0.625,0.770075


In this section we import the metrics we want to use in our model

> **Important note**

 since we have used the whole development dataset to train our model, there is no 'test_X' or 'test_y' based on this dataset.<br>
we also don't have the true labels for the evaluation dataset, so we can't print the r2_score of our model in here.<br> 
the only thing we can do is to show our model's r2_score on the dev dataset.<br>
altought we have used the whole dataset to train our model and our whole dataset is exposed to our model, but it gives us a grisp of our model's performance.

In [30]:
#we create a function to get the output as a dataframe and alos print the r2_score for our model
def predict_on_eval(model, data, name):
  eval_predict = model.predict(data)
  eval_predict_df = pd.DataFrame(eval_predict)
  eval_predict_df.index.name = 'Id'
  eval_predict_df.to_csv(name + '.csv', header=['Predicted'])
  print(Counter(eval_predict))
  return eval_predict_df


def get_metrics(model, data, true_labels):
  model_prediction = model.predict(data)
  return r2_score(true_labels,model_prediction)

Now we use different models on out data set




We have to visualize data in order to have a good grisp on the data to undrestand it 

In [31]:
pip install dash

In [32]:
# #we're gonna use plotly to see our data 
import plotly.express as px
fig = px.histogram(y, x="review/overall")
fig.show()

I used used [plotly](https://plotly.com/) to visualize the data <br>
Our data is somehow **imbalanced**<br>
we can either use undersampling or oversampling but there is no gurantie that they work 

In [33]:
!pip install imbalanced-learn

In [34]:
# we run undersampling and get a undersampled data
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_rus, y_rus = rus.fit_resample(X, y)
print(sorted(Counter(y_rus).items()))

[(1.0, 483), (1.5, 483), (2.0, 483), (2.5, 483), (3.0, 483), (3.5, 483), (4.0, 483), (4.5, 483), (5.0, 483)]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:93: UserWarning:

'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got continuous instead.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in ve

In [35]:
#we're gonna use plotly to see our data 
import plotly.express as px
fig = px.histogram(y_rus, x=y_rus)
fig.show()

In [36]:
# we perform oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_ros, y_ros = ros.fit_resample(X, y)
print(sorted(Counter(y_ros).items()))


[(1.0, 25711), (1.5, 25711), (2.0, 25711), (2.5, 25711), (3.0, 25711), (3.5, 25711), (4.0, 25711), (4.5, 25711), (5.0, 25711)]


/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:93: UserWarning:

'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got continuous instead.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



In [37]:
# #we're gonna use plotly to see our data 
import plotly.express as px
fig = px.histogram(y_ros,x=y_ros)
fig.show()

Oversampling looks fine
but Undersampling doesn't, best idea for the undersampling is to reduce the number of 4 values by 10k in order to balance the data

Now our data is ready.<br>
we have **X** and **y** which are the whole dev dataset and we're gonna use them to train our final model <br>
we have **X_rus** and **y_rus** which are the undersampled data (performed on the whole dev dataset) <br>
we have **X_ros** and **y_ros** which are the oversampled data (performed on the whole dev dataset) <br>

We also have to use the "review/text" column as well, so we use the tf-idf technique 

In [38]:
%%time
review_text_dev=review_text_dev.astype(str).values.tolist()
from sklearn.feature_extraction.text import TfidfVectorizer
vec=TfidfVectorizer(stop_words="english", binary=True, use_idf=True,norm=False,min_df=1000)
tfidf_vec=vec.fit_transform(review_text_dev)

CPU times: user 6.56 s, sys: 117 ms, total: 6.68 s
Wall time: 6.72 s


In [39]:
#we have to change the shapes inorder to make it possible to concatinate the numpy arrays(X and tf_idf )
final_tfidf=[]
for i in tfidf_vec:
  final_tfidf.append(i.toarray()[0])

final_tfidf=np.array(final_tfidf)

#now we concatenate other columns to our tf-idf column
final_X=np.concatenate((X.to_numpy(),final_tfidf), axis=1)

In [40]:
final_tfidf.shape

(70000, 623)

In [41]:
final_X.shape

(70000, 631)

In [42]:
%%time
review_text_eval=review_text_eval.astype(str).values.tolist()
tfidf_vec_eval=vec.transform(review_text_eval)

CPU times: user 2.6 s, sys: 26 ms, total: 2.62 s
Wall time: 2.64 s


In [43]:
final_tfidf_eval=[]
for i in tfidf_vec_eval:
  final_tfidf_eval.append(i.toarray()[0])

final_tfidf_eval=np.array(final_tfidf_eval)

final_eval_dataset=np.concatenate((eval_dataset.to_numpy(),final_tfidf_eval), axis=1)

In [44]:
final_tfidf_eval.shape

(30000, 623)

In [45]:
final_eval_dataset.shape

(30000, 631)

# Random Forrest Regression

In [46]:
%%time
# first we run our model without under/over sampling 
RFG = RandomForestRegressor(100, random_state=42)
RFG.fit(final_X,y)#we train the model on our training set
#there is no test set since we want to use all of our data
#that's the reason we don't use the r2_score in the code and we leave it to the platform the give us the r2_score of our model
# or we can use the traditional model and split our data
predict_on_eval(RFG,final_eval_dataset,'RandomForestRegressor')

Counter({4.115: 183, 4.17: 175, 4.03: 175, 4.045: 172, 4.095: 170, 4.1: 170, 4.005: 169, 4.125: 169, 4.175: 164, 4.205: 163, 4.195: 162, 4.05: 162, 4.145: 161, 4.06: 161, 4.02: 160, 4.14: 160, 4.2: 160, 4.105: 159, 4.065: 159, 4.16: 159, 4.18: 159, 4.08: 158, 4.04: 155, 4.12: 155, 4.025: 155, 4.015: 154, 4.13: 153, 4.075: 152, 4.135: 152, 4.055: 152, 4.155: 150, 4.07: 150, 3.995: 150, 4.15: 149, 4.11: 149, 4.01: 149, 4.085: 147, 4.0: 145, 4.035: 145, 4.165: 143, 4.215: 142, 3.975: 141, 4.23: 139, 4.09: 139, 3.97: 138, 3.94: 138, 3.96: 136, 4.185: 136, 4.22: 132, 4.255: 129, 3.98: 129, 4.225: 129, 3.985: 129, 3.965: 128, 4.245: 128, 4.24: 126, 4.28: 126, 4.235: 125, 3.9: 124, 3.95: 123, 4.27: 123, 3.99: 122, 3.935: 121, 4.21: 121, 4.265: 121, 4.295: 121, 4.275: 120, 3.925: 120, 4.26: 120, 3.865: 119, 4.25: 118, 3.93: 118, 3.905: 116, 3.945: 116, 3.845: 116, 3.665: 115, 3.88: 115, 3.775: 115, 3.735: 113, 3.885: 112, 3.825: 112, 3.74: 111, 3.955: 111, 3.86: 111, 3.765: 110, 3.915: 110, 4.

In [47]:
# #getting the r2_score value for our model
# print('Training data prediction results:')
# get_metrics(RFG, X, y)
# r2_score on the submittion portal : 0.690


In [48]:
# Random Forrest Regression with Random undersampling
# RFG_rus = RandomForestRegressor(100, random_state=42)
# RFG_rus.fit(X_rus,y_rus)
# predict_on_eval(RFG,eval_dataset,'RandomForestRegressor_rus')

In [49]:
# #getting the r2_score value for our model
# print('Training data prediction results:')
# get_metrics(RFG_rus, X_rus, y_rus)
# r2_score on the submittion portal : 0.57

In [50]:
# Random Forrest Regression with Random oversampling
# RFG_ros = RandomForestRegressor(100, random_state=42)
# RFG_ros.fit(X_ros,y_ros)
# predict_on_eval(RFG_ros,eval_dataset,'RandomForestRegressor_ros')

In [51]:
# #getting the r2_score value for our model
# print('Training data prediction results:')
# get_metrics(RFG_ros, X_ros, y_ros)
#without using tf-idf 

In [52]:
# RFG = RandomForestRegressor(100, random_state=50,criterion='mse')
# RFG.fit(X,y)
# predict_on_eval(RFG,eval_dataset,'RandomForestRegressor_new')
#without using tf-idf 

without grid search, we tested oversampling and undersampling on the dev dataset but it was not a good idea. <br>
undersampling didn't work right beacuase we don't have much data to begin with <br>
oversampling gave the same results as the data without any sampling.<br>
This part was only an experiment to see whether the sampling methods help or not.

Now we performe **Gridsearch** for Random forrest regression
> **Note**<br>
there are other libraries to performe grid search, libraries like [hypopt](https://pypi.org/project/hypopt/). <br>
I used it but it interfeard with some sklearn libraries so I avoided using it

In [53]:
# param_grid_RGG=[{'criterion':['mse','mae'],'random_state':[0,25,50]}]
# grid_search_RGG=GridSearchCV(RandomForestRegressor(),param_grid_RGG,scoring='r2',n_jobs=-1,cv=None,verbose=3)
# grid_search_RGG.fit(X, y)
# grid_search_RGG.best_params_

GridSearch is a time consuming operation and it takes lots of resources and not all Laptops and PCs can performe such computation.
So instead I ran the models one by one.
I ran the Project both on colab and also my local machine and it didn't work.
So I'm gonna check different models one by one.

In [54]:

# RFG = RandomForestRegressor(100, random_state=50,criterion='mae')
# RFG.fit(X,y)
# predict_on_eval(RFG,eval_dataset,'RandomForestRegressor_mae')


The cell above was taking so much to run on my local machine, so I ran the code on colab and even colab didn't give an answer.<br>
So we can easily realize that it is not a feasible choice of parameters.<br>

We can realize that the default hyperparamethers for Random Forrest Regressor is the best one we've got

## **SVR**

The Grid Search procedure was done before using the tf-idf technique.<br>
good parameters have been gained but still it didn't converge after adding tf-idf. 

In [55]:
%%time
# param_grid_LSVR=[{'loss':['epsilon_insensitive','squared_epsilon_insensitive'],'C':[1,10,100]}]
# grid_search_LSVR=GridSearchCV(LinearSVR(max_iter=20000),param_grid_LSVR,scoring='r2',n_jobs=-1,cv=None,verbose=3)
# grid_search_LSVR.fit(X, y)
# grid_search_LSVR.best_params_

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [59]:
# now we test svm
%%time
LSVR=LinearSVR(C=1.0, epsilon=0.2,max_iter=5000)
LSVR.fit(final_X,y)
predict_on_eval(LSVR,final_eval_dataset,'Linear_SVR')

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



Counter({2.29287426615055: 1, 4.065069440088705: 1, 3.765204224120067: 1, 4.535073966750456: 1, 2.4044967065568805: 1, 3.5575468417248466: 1, 3.57789655921347: 1, 3.826705506328987: 1, 3.2617259330083748: 1, 4.260593027466717: 1, 4.0596686865553515: 1, 3.853772951561469: 1, 2.8417509871929294: 1, 4.223098013029785: 1, 4.294642110887125: 1, 3.9326239799478726: 1, 4.119077738947909: 1, 4.29979827860076: 1, 4.6133389785351016: 1, 4.357849294107989: 1, 3.531978793601378: 1, 4.23872994466001: 1, 3.2293991453508157: 1, 4.653145458752928: 1, 2.245163383679187: 1, 3.5747206754675656: 1, 4.638595465788912: 1, 3.4203837111203264: 1, 4.008148066394938: 1, 4.093352936344844: 1, 2.975926790423104: 1, 3.825727613291277: 1, 4.150909771141044: 1, 3.764228465707829: 1, 4.150963630776701: 1, 3.6079867997625987: 1, 4.176835204391178: 1, 4.193163368141462: 1, 2.739452636684276: 1, 3.9453109955202343: 1, 3.9765540778706505: 1, 3.9891593124116342: 1, 4.344150738412021: 1, 4.209634477955083: 1, 3.64720311883

Grid seearch for Linear SVR worked on local machine and I got the best combinations. but when using the "review/text" column, it won't converge due to high number of vectors, so it's useless

## **Ridge**

In [60]:
#output value: 0.697

In [61]:
#finding the  best value
%%time
from sklearn.linear_model import Ridge
param_grid_Ridge=[{'alpha':[1.0,0.5,0.75,0.25]}]
grid_search_Ridge=GridSearchCV(Ridge(),param_grid_Ridge,scoring='r2',n_jobs=-1,cv=None,verbose=3)
grid_search_Ridge.fit(final_X, y)
grid_search_Ridge.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   30.4s finished


CPU times: user 3.06 s, sys: 1.37 s, total: 4.43 s
Wall time: 32.3 s


In [62]:
grid_search_Ridge.best_params_

{'alpha': 0.25}

In [63]:
from sklearn.linear_model import Ridge
Ridge=Ridge(alpha=0.25)
Ridge.fit(final_X,y) 
predict_on_eval(Ridge,final_eval_dataset,'Ridge_final')

Counter({2.295541431484633: 1, 4.01793437424173: 1, 3.768119805006277: 1, 4.582789988778568: 1, 2.4479255244358478: 1, 3.5020101408734057: 1, 3.5820327307532676: 1, 3.8607576358605367: 1, 3.3071237886820724: 1, 4.21201711035515: 1, 4.072979545123585: 1, 3.8957672450899334: 1, 2.9602132047160747: 1, 4.203008880229007: 1, 4.340220098451795: 1, 3.9486838917812612: 1, 4.148325548214132: 1, 4.189686809493748: 1, 4.603725597979077: 1, 4.420039984507085: 1, 3.548254390327748: 1, 4.211681020513206: 1, 3.270562101678439: 1, 4.660687106045874: 1, 2.290846369723145: 1, 3.5401871546384402: 1, 4.669873119192763: 1, 3.4166107487155735: 1, 4.00695807491292: 1, 4.0767349270822795: 1, 2.968257573836328: 1, 3.831668371680123: 1, 4.174510169609013: 1, 3.791587344538831: 1, 4.0768262291954525: 1, 3.673274187609578: 1, 4.169075826193417: 1, 4.233474443007462: 1, 2.7790184512024148: 1, 3.9669147803817255: 1, 3.976774683043602: 1, 4.024472017094933: 1, 4.3183023022145814: 1, 4.195622467547375: 1, 3.694925425

,0
Id,
0,2.295541
1,4.017934
2,3.768120
3,4.582790
4,2.447926
...,...
29995,3.117953
29996,3.704236
29997,3.071673


The best r2_score for ridge is **0.697**